# Лекция 9: метаклассы, тестирование

## Метаклассы

Metaclasses are deeper magic than 99% of users should
never worry about. If you wonder whether you need them,
you don’t (the people who actually need them know with
certainty that they need them, and don’t need an explanation
about why).

by Tim Peters


## Классы и классы классов

Помним, что в Python всё является объектом:

In [1]:
print(type(1))

<class 'int'>


In [2]:
class MyInt(int):
    def __add__(self, other):
        print("specializing addition")
        return super(MyInt, self).__add__(other)

i = MyInt(2)
print(i + 2)

specializing addition
4


Классы также являются объектами.

In [3]:
class Empty:
    pass

Воспользуемся встроенной функцией type() для определения типа объекта:

In [4]:
obj = Empty()
print(type(obj))

<class '__main__.Empty'>


Проверим тип класса:

In [5]:
type(Empty)

type

Oh, wait... What if?

In [6]:
print(type(tuple), type(list), type(int), type(float))

<class 'type'> <class 'type'> <class 'type'> <class 'type'>


* Классы сами по себе являются объектами типа type.  

Hmmm...

In [7]:
print(type(type))

<class 'type'>


* type является предком себя - такую циклическую зависимость нельзя сделать в чистом Python, поэтом она реализуется  небольшим хаком в реализации Python.
* type имеет несколько значений в Python:
  * type() - как функция возвращая тип переданного ей объекта.
  * type - как конструктор типов.
  * type - как корневой тип для метаклассов, предок всех классов (подробности ниже).

## Метапрограммирование

* В общем случае, программы которые порождают другие программы или динамически модифицируются.
* В частном случае, возможность динамического создания новых типов во время выполнения.

Мы можем написать функцию, динамически создающую объекты:

In [8]:
def int_factory(s):
    i = int(s)
    return i

i = int_factory('100')
print(i)

100


Это простейший пример функции-фабрики, которая принимает некоторые аргументы, конструирует нужный объект и возвращает его.

* Ничего не мешает нам создать функцию, которая будет создавать объект типа type (т.е. некоторый класс) и возвращать его в качестве результата
* Такая функция называется метафункцией.

In [9]:
def class_factory():
    class Foo:
        pass
    return Foo

F = class_factory()
f = F()
print(type(f))

<class '__main__.class_factory.<locals>.Foo'>


* Получившаяся функция создаёт класс, однако выглядит несколько странно.  
* Было бы отлично избежать явного написания кода и выполнять эти действия более динамически.
* Это можно сделать создавая наш класс напрямую от класса type.

In [10]:
def class_factory():
    return type('Foo', (), {})

F = class_factory()
f = F()
print(type(f))

<class '__main__.Foo'>


Вообще, запись вида:

In [11]:
class Empty:
    pass

Эквивалентна следующей конструкции:

In [12]:
Empty = type('Empty', (), {})

### type - конструктор типов

type(name, bases, dct)

* name - имя для создаваемого класса.
* bases - tuple задающий родительские классы для конструируемого класса.
* dct - словарь аттрибутов и методов создаваемого класса.

Следующие примеры дадут идентичные результаты:

In [13]:
class Foo:
    i = 4

class Bar(Foo):
    def get_i(self):
        return self.i
    
b = Bar()
print(b.get_i())

4


In [14]:
Foo = type('Foo', (), dict(i=4))

Bar = type('Bar', (Foo,), dict(get_i = lambda self: self.i))

b = Bar()
print(b.get_i())

4


### type как корневой тип для метаклассов

Мы можем создать собственный метакласс отнаследовавшись от type, а затем использовать его вместо type при создании классов.

При создании нового класса с помощью ключевого слова class происходит следующее:  
* Метакласс определяется путём поиска у себя или у предков keyword-аргумента metaclass для класса.
* Подготавливается namespace-словарь перед выполнением тела класса для того, чтобы поля добавлялись в него. Для этого используется опциональный метод (classmethod) \_\_prepare\_\_ у метакласса.
* Python выполняет тело внутри конструкции class как обычный блок кода.
* Получившийся дополненный namespace (словарь) содержит в себе аттрибуты будущего класса.
* Найденный метакласс вызывается, ему передаётся имя, список базовых классов и аттрибуты создаваемого класса.

Такой двухэтапный подход позволяет более гибко влиять на создание класса.

### Пример метакласса - сохраняем порядок полей

In [15]:
import collections

class OrderedClass(type):

    @classmethod
    def __prepare__(metacls, name, bases, **kwargs):
        return collections.OrderedDict()

    def __new__(cls, name, bases, namespace, **kwargs):
        result = type.__new__(cls, name, bases, dict(namespace))
        result.members = tuple(namespace)
        return result

class A(metaclass=OrderedClass):
    def one(self):
        pass
    def two(self):
        pass
    def three(self):
        pass
    def four(self):
        pass

In [16]:
A.members

('__module__', '__qualname__', 'one', 'two', 'three', 'four')

### Пример метакласса - регистрируем подклассы

Следующий метакласс просто добавляет словарь registry (реестр) и каждый раз при создании нового класса кладёт в registry имя этого класса.

In [17]:
class DBInterfaceMeta(type):
    # we use __init__ rather than __new__ here because we want
    # to modify attributes of the class *after* they have been
    # created
    def __init__(cls, name, bases, dct):
        if not hasattr(cls, 'registry'):
            # this is the base class.  Create an empty registry
            cls.registry = {}
        else:
            # this is a derived class.  Add cls to the registry
            interface_id = name.lower()
            cls.registry[interface_id] = cls

        super(DBInterfaceMeta, cls).__init__(name, bases, dct)

Мы можем задать классу другой (вместо type) метакласс путём присваивания аргументу metaclass:

In [18]:
class DBInterface(metaclass=DBInterfaceMeta):
    pass
    
print(DBInterface.registry)

{}


После добавления подклассов:

In [19]:
class FirstInterface(DBInterface):
    pass

class SecondInterface(DBInterface):
    pass

class SecondInterfaceModified(SecondInterface):
    pass

print(DBInterface.registry)

{'secondinterfacemodified': <class '__main__.SecondInterfaceModified'>, 'firstinterface': <class '__main__.FirstInterface'>, 'secondinterface': <class '__main__.SecondInterface'>}


### Составной пример метакласса

В Python существует два метода, которые используются при создании объекта (в т.ч. классов и метаклассов):  
* \_\_new\_\_ - конструктор, вызывается непосредственно для создания объекта, редко когда нужен (если нам надо иметь  контроль над созданием объекта).
* \_\_init\_\_ - инициализатор, вызывается после создания объекта для его инициализации, в основном используется этот метод.

В случае с метаклассами редко, но может иметь смысл перегрузка функции \_\_call\_\_ в метаклассе (вызывается, когда идёт создание объекта с помощью имени класса MyClass()).

По сути, эти три функции позволяют влиять на разные этапы создания класса и его объектов.

In [20]:
def make_hook(f):
    """Decorator to turn 'foo' method into '__foo__'"""
    f.is_hook = 1
    return f

class MyType(type):
    def __new__(cls, name, bases, attrs):
        print("__new__ for {}".format(name))
        if name.startswith('None'):
            return None

        # Go over attributes and see if they should be renamed.
        newattrs = {}
        for attrname, attrvalue in attrs.items():
            if getattr(attrvalue, 'is_hook', 0):
                newattrs['__%s__' % attrname] = attrvalue
            else:
                newattrs[attrname] = attrvalue

        return super(MyType, cls).__new__(cls, name, bases, newattrs)

    def __init__(self, name, bases, attrs):
        super(MyType, self).__init__(name, bases, attrs)

        print("__init__ for {}".format(self))

    def __add__(self, other):
        class AutoClass(self, other):
            pass
        return AutoClass
        # Alternatively, to autogenerate the classname as well as the class:
        # return type(self.__name__ + other.__name__, (self, other), {})

    def say_hello(self):
        print("Hello, {}".format(self))

In [21]:
class MyObject(metaclass=MyType):
    pass

__new__ for MyObject
__init__ for <class '__main__.MyObject'>


In [22]:
class NoneSample(MyObject):
    pass

# Will print "NoneType None"
print(type(NoneSample), repr(NoneSample))

__new__ for NoneSample
<class 'NoneType'> None


In [23]:
class Example(MyObject):
    def __init__(self, value):
        self.value = value

    @make_hook
    def add(self, other):
        return self.__class__(self.value + other.value)

Example.say_hello()

__new__ for Example
__init__ for <class '__main__.Example'>
Hello, <class '__main__.Example'>


In [24]:
inst = Example(10)

print(inst + inst)

In [25]:
class Sibling(MyObject):
    pass

ExampleSibling = Example + Sibling
# ExampleSibling is now a subclass of both Example and Sibling (with no
# content of its own) although it will believe it's called 'AutoClass'
print(ExampleSibling)
print(ExampleSibling.__mro__)

__new__ for Sibling
__init__ for <class '__main__.Sibling'>
__new__ for AutoClass
__init__ for <class '__main__.MyType.__add__.<locals>.AutoClass'>
<class '__main__.MyType.__add__.<locals>.AutoClass'>
(<class '__main__.MyType.__add__.<locals>.AutoClass'>, <class '__main__.Example'>, <class '__main__.Sibling'>, <class '__main__.MyObject'>, <class 'object'>)


### Дополнительное чтение про метаклассы

* https://docs.python.org/3.5/reference/datamodel.html#metaclasses - в деталях про этапы создания класса.
* https://www.python.org/dev/peps/pep-3115/ - PEP про метаклассы в python 3.x
* http://eli.thegreenplace.net/2011/08/14/python-metaclasses-by-example - местами более подробная версия этой лекции, но в варианте для python 2.x, с дополнительными примерами кода и применения в библиотеках. Большая часть актуальна и для 3.x.
* http://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/ - подробная статья, в деталях соединяющая (с картинками) материал этой лекции и одной из предыдущих про аттрибуты. Версия для python 2.x, но большая часть актуальна и для 3.x.  

## Тестирование и Python

### Пару слов про тестирование и программирование.

Тестирование как способ проверить работу программы или её части в заданных условиях.

#### Некоторые типы тестирования

* юнит
* модульное
* интеграционное
* регрессионное
* нагрузочное
* fuzzy

#### Порядок среди хаоса

* Интерфейсы и соглашения об ожидаемом поведении кода
* Фиксация "контрактов" на поведение через тесты

### Почему нужно писать тесты

* Python - динамически типизированный язык программирования, нет привычной по другим языкам статической проверки типов.
* Удобно авматоматизировать ручные проверки старых и новых функций и их взаимодействия.
* При внесении изменений в код хотелось бы иметь простой способ быстро убедиться, что ничего не сломалось.
* При работе в команде это способ отслеживания состояния и работоспособности проекта.
* Тесты - некоторого рода документация, описывающая как взаимодействовать с кодом.

### Юнит-тестирование

* Существует ряд различных типов тестирования, которые применяются так или  иначе для проверки отдельных аспектов работы программы.
* Нас интересует один из часто используемых - юнит-тестирование.
* Юнит-тестирование (unit testing) предполагает написание тестов для небольших частей кода (функций или классов).
* В таких тестах проверяется выполнение ожидаемых инвариантов работы с помощью различных вариантов функции assert.
* Проверяем: что вернулось ожидаемое значение, что два значения равны, что было выброшено (или не выброшено) исключение,  что вызов завершился до истечения таймаута и т.п.

### Python unittest

* В результате развития подходом и библиотек для языка Java в какой-то момент появилась успешная библиотека JUnit.
* unittest - реализации идей этой библиотеки для языка Python.

Основные концепции выполнения тестов:

* создание начальных условий для отдельного теста
* задание завершающих действий для отдельного теста
* объединение тестов в группы
* автоматизированный запуск наборов тестов
* независимое отображение результатов выполнения тестов

Для реализации этих концепций используются понятия:

* Отдельный тест (test case, класс TestCase) - описывает действия подготовки и завершения (setUp и tearDown), а также сам тест.
* Набор тестов (test suite, TestSuite) - объединяет в себе несколько тестов.
* Тестовые установки (test fixture, TestFixture) - представляют действия для подготовки и завершения одного или нескольких тестов (создание временных таблиц, директорий и т.п.)
* Исполнитель тестов (test runner, TestRunner) - компонент, отвечающий за запуск тестов и отображения их результатов (текстовы, графический интерфейсы).

Пример отдельного теста:

In [26]:
import unittest

class TestStringMethods(unittest.TestCase):

  def test_upper(self):
      self.assertEqual('foo'.upper(), 'FOO')

  def test_isupper(self):
      self.assertTrue('FOO'.isupper())
      self.assertFalse('Foo'.isupper())

  def test_split(self):
      s = 'hello world'
      self.assertEqual(s.split(), ['hello', 'world'])
      # check that s.split fails when the separator is not a string
      with self.assertRaises(TypeError):
          s.split(2)

In [27]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestStringMethods)
unittest.TextTestRunner(verbosity=2).run(suite)

test_isupper (__main__.TestStringMethods) ... ok
test_split (__main__.TestStringMethods) ... ok
test_upper (__main__.TestStringMethods) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [28]:
import unittest

class WidgetTestCase(unittest.TestCase):
    def setUp(self):
        self.widget = Widget('The widget')

    def tearDown(self):
        self.widget.dispose()
        self.widget = None

    def test_default_size(self):
        self.assertEqual(self.widget.size(), (50,50),
                         'incorrect default size')

    def test_resize(self):
        self.widget.resize(100,150)
        self.assertEqual(self.widget.size(), (100,150),
                         'wrong size after resize')

In [29]:
import unittest

class SimpleWidgetTestCase(unittest.TestCase):
    def setUp(self):
        self.widget = Widget('The widget')

class DefaultWidgetSizeTestCase(SimpleWidgetTestCase):
    def runTest(self):
        self.assertEqual(self.widget.size(), (50,50),
                         'incorrect default size')

class WidgetResizeTestCase(SimpleWidgetTestCase):
    def runTest(self):
        self.widget.resize(100,150)
        self.assertEqual(self.widget.size(), (100,150),
                         'wrong size after resize')


In [ ]:
class MyTestCase(unittest.TestCase):

    @unittest.skip("demonstrating skipping")
    def test_nothing(self):
        self.fail("shouldn't happen")

    @unittest.skipIf(mylib.__version__ < (1, 3),
                     "not supported in this library version")
    def test_format(self):
        # Tests that work for only a certain version of the library.
        pass

    @unittest.skipUnless(sys.platform.startswith("win"), "requires Windows")
    def test_windows_support(self):
        # windows specific testing code
        pass

Предлагается самостоятельно почитать подробнее, изучить примеры и попробовать писать тесты:

https://docs.python.org/3.5/library/unittest.html - официальная документация по библиотеке unittest и ссылки на альтернативы.

### Библиотеки для юнит-тестов

Существует также ряд других библиотек, которые могут быть интересны в будущем:

* doctest - выполнение тестов особого вида, помещённых прямо в docstring.
* pytest - альтернатива стандартному unittest, может подкупить простым интерфейсом, есть удобная параметризация тестов.
* nose - расширяет unittest упрощая тестирование (test discovery, показатель test coverage).
* tox - автоматизация управления тестовым окружением, позволяет задавать матрицы параметров для тестов.
* mock - библиотека для тестирования, позволяет просто создавать объекты-заглушки для тестирования и заменять части системы на время тестирования (см. mock тестирование).

In [30]:
"""
This is the "example" module.

The example module supplies one function, factorial().  For example,

>>> factorial(5)
120
"""

def factorial(n):
    """Return the factorial of n, an exact integer >= 0.

    >>> [factorial(n) for n in range(6)]
    [1, 1, 2, 6, 24, 120]
    >>> factorial(30)
    265252859812191058636308480000000
    >>> factorial(30)
    265252859812191058636308480000000
    >>> factorial(-1)
    Traceback (most recent call last):
        ...
    ValueError: n must be >= 0

    Factorials of floats are OK, but the float must be an exact integer:
    >>> factorial(30.1)
    Traceback (most recent call last):
        ...
    ValueError: n must be exact integer
    >>> factorial(30.0)
    65252859812191058636308480000000

    It must also not be ridiculously large:
    >>> factorial(1e100)
    Traceback (most recent call last):
        ...
    OverflowError: n too large
    """

    import math
    if not n >= 0:
        raise ValueError("n must be >= 0")
    if math.floor(n) != n:
        raise ValueError("n must be exact integer")
    if n+1 == n:  # catch a value like 1e300
        raise OverflowError("n too large")
    result = 1
    factor = 2
    while factor <= n:
        result *= factor
        factor += 1
    return result


if __name__ == "__main__":
    import doctest
    doctest.testmod()

**********************************************************************
File "__main__", line 29, in __main__.factorial
Failed example:
    factorial(30.0)
Expected:
    65252859812191058636308480000000
Got:
    265252859812191058636308480000000
**********************************************************************
1 items had failures:
   1 of   7 in __main__.factorial
***Test Failed*** 1 failures.


### Общие советы по написанию тестов

* Обычно тесты кладут в папку tests в проекте.
* Удобно использовать по одному файлу с тестами на каждый модуль.
* Стараться избавляться от избыточной дупликации кода в отдельных тестах, если это можно сделать.
* Отдельный тест должен проверять небольшую часть функциональности (если пройден, то она верна).
* Каждый тест должен быть полностью независимым.
* Тесты должны работать быстро, иначе будут проблемы с их частым и массовым запуском.
* Удобно, когда на каждый коммит в репозиторий автоматически запускаются тесты.
* Если есть долгие тесты, то можно их запускать по расписанию, а не при каждом коммите.
* Полезно писать как можно подробные имена тестов.